In [18]:
import pandas as pd
import matplotlib

In [19]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func 

In [39]:
funding_csv = "funding_types.csv"
funding_df_2014 = pd.read_csv(funding_csv)
# delete last column
# del funding_df_2014["Unnamed: 13"]
# # dropping extraneous first two rows
funding_df_2014 = funding_df_2014.dropna(how='any')

# # making "Row Labels" as the column head
header = funding_df_2014.iloc[0]
funding_df_2014= funding_df_2014[1:] 
funding_df_2014 = funding_df_2014.rename(columns=header)
# # making 'row labels' as the index
funding_df_2014 = funding_df_2014.set_index("Row Labels")
funding_df_2014 = funding_df_2014.drop(["Profiles: Organizations", "Profiles: People", 
                              "Profiles: Products", "Profiles"])
funding_df_2014

,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,total
Row Labels,,,,,,,,,,,,,,
Rounds: angel,106,68,118,106,133,93,115,142,220,116,120,95,90,"1,522"
Rounds: seed,584,342,521,918,"1,368",644,"1,405",825,737,901,"1,155",808,602,"10,810"
Rounds: venture,"1,524","1,647","1,875","1,826","1,407",879,985,"1,225","1,062",740,740,636,743,"15,289"
Rounds: convertible_note,47,26,41,52,41,19,32,31,121,32,71,25,67,605
Rounds: debt_financing,361,411,361,458,292,141,125,111,204,132,106,87,141,"2,930"
Rounds: grant,57,52,49,57,40,36,131,44,109,61,82,49,27,794
Rounds: post_ipo_equity,15,10,15,35,51,19,31,31,17,31,26,19,14,314
Rounds: private_equity,42,40,44,51,36,46,74,72,40,41,46,60,38,630
Rounds: product_crowdfunding,13,16,25,10,7,4,13,11,14,13,13,10,7,156


In [208]:
funding_2008 = "funding_2008.csv"
funding_df_2008 = pd.read_csv(funding_2008)

# making "Row Labels" as the column head
header = funding_df_2008.iloc[2] 
funding_df_2008= funding_df_2008[1:] 
funding_df_2008 = funding_df_2008.rename(columns=header)
# making 'row labels' as the index
funding_df_2008 = funding_df_2008.set_index("Row Labels") 
funding_df_2008 = funding_df_2008.drop(["Profiles: Organizations", "Profiles: People", 
                              "Profiles: Products", "Profiles", "Sum of value", "Row Labels" ])
funding_df_2008

,2008-01,2008-02,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12,total
Row Labels,,,,,,,,,,,,,
Rounds: angel,13,77,34,31,33,46,27,25,27,26,28,34,401
Rounds: seed,13,64,57,35,36,64,55,59,45,41,49,45,563
Rounds: venture,131,137,135,120,184,195,131,129,100,116,107,219,"1,704"
Rounds: convertible_note,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
Rounds: debt_financing,3,3,NaN,1,8,3,7,7,1,2,6,5,46
Rounds: grant,1,NaN,NaN,NaN,NaN,1,1,NaN,NaN,1,1,NaN,5
Rounds: post_ipo_equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,NaN,NaN,NaN,4
Rounds: private_equity,1,NaN,NaN,NaN,2,1,3,1,4,NaN,NaN,5,17
Rounds: undisclosed,NaN,NaN,1,2,4,10,5,10,6,3,4,7,52


In [202]:
funding_2009 = "funding_2009.csv"
funding_df_2009 = pd.read_csv(funding_2009)

# making "Row Labels" as the column head
header = funding_df_2009.iloc[2]
funding_df_2009= funding_df_2009[1:] 
funding_df_2009 = funding_df_2009.rename(columns=header)
# making 'row labels' as the index
funding_df_2009 = funding_df_2009.set_index("Row Labels")
funding_df_2009 = funding_df_2009.drop(["Profiles: Organizations", "Profiles: People", 
                              "Profiles: Products", "Profiles", "Sum of value", "Row Labels"])
funding_df_2009

,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12,total
Row Labels,,,,,,,,,,,,,
Rounds: angel,35,19,16,20,14,26,20,32,34,22,23,29,290
Rounds: seed,49,36,74,50,59,60,43,83,89,61,52,59,715
Rounds: venture,461,106,149,187,305,606,586,280,399,237,299,615,"4,230"
Rounds: convertible_note,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3
Rounds: debt_financing,10,3,4,8,6,11,10,7,15,6,24,66,170
Rounds: grant,NaN,NaN,NaN,1,NaN,1,2,7,12,6,17,23,69
Rounds: post_ipo_debt,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,1,NaN,3
Rounds: post_ipo_equity,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,1,1,3,7
Rounds: private_equity,4,2,1,2,4,18,1,NaN,5,2,6,13,58


In [198]:
funding_2010 = "funding_2010.csv"
funding_df_2010 = pd.read_csv(funding_2010)


# making "Row Labels" as the column head
header = funding_df_2010.iloc[2]
funding_df_2010= funding_df_2010[1:] 
funding_df_2010 = funding_df_2010.rename(columns=header)
# # making 'row labels' as the index
funding_df_2010 = funding_df_2010.set_index("Row Labels")
funding_df_2010 = funding_df_2010.drop(["Profiles: Organizations", "Profiles: People", 
                              "Profiles: Products", "Profiles", "Sum of value", "Row Labels"])
funding_df_2010

,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,2010-10,2010-11,2010-12,total
Row Labels,,,,,,,,,,,,,
Rounds: angel,29,20,35,26,25,31,26,25,34,48,41,26,366
Rounds: seed,50,62,76,74,77,74,79,96,80,89,88,69,914
Rounds: venture,595,550,880,"1,160",567,455,607,412,418,822,317,231,"7,014"
Rounds: convertible_note,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,3
Rounds: debt_financing,58,31,48,39,17,14,10,9,13,10,17,10,276
Rounds: grant,22,4,11,5,4,6,10,11,13,18,13,4,121
Rounds: post_ipo_debt,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,2
Rounds: post_ipo_equity,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,5
Rounds: private_equity,14,11,13,8,6,6,15,3,3,8,6,3,96


In [191]:
funding_2011 = "funding_2011.csv"
funding_df_2011 = pd.read_csv(funding_2011)

# # # dropping extraneous first two rows
# funding_df_2011 = funding_df_2011.dropna(how='any') 

# # # # # making "Row Labels" as the column head
header = funding_df_2011.iloc[2]
funding_df_2011= funding_df_2011[1:] 
funding_df_2011 = funding_df_2011.rename(columns=header)
# # # making 'row labels' as the index
funding_df_2011 = funding_df_2011.set_index("Row Labels")
funding_df_2011 = funding_df_2011.drop(["Profiles: Organizations", "Profiles: People", 
                              "Profiles: Products", "Profiles", "Sum of value", "Row Labels"])
funding_df_2011

,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,total
Row Labels,,,,,,,,,,,,,
Rounds: angel,39,34,39,49,52,42,56,48,51,34,49,29,522
Rounds: seed,71,89,133,105,121,133,130,115,125,113,137,107,"1,379"
Rounds: venture,273,242,252,226,261,295,248,285,269,257,243,228,"3,079"
Rounds: convertible_note,NaN,NaN,NaN,NaN,1,1,1,NaN,NaN,1,NaN,NaN,4
Rounds: debt_financing,13,13,9,8,12,12,10,14,16,21,10,19,157
Rounds: grant,10,11,4,5,11,9,5,19,11,9,9,14,117
Rounds: post_ipo_equity,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
Rounds: private_equity,10,3,7,8,2,9,13,6,9,4,6,6,83
Rounds: product_crowdfunding,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,2


In [182]:
funding_2012 = "funding_2012.csv"
funding_df_2012 = pd.read_csv(funding_2012)

# making "Row Labels" as the column head
header = funding_df_2012.iloc[2]
funding_df_2012= funding_df_2012[1:] 
funding_df_2012 = funding_df_2012.rename(columns=header)
# # # making 'row labels' as the index
funding_df_2012 = funding_df_2012.set_index("Row Labels")
funding_df_2012 = funding_df_2012.drop(["Profiles: Organizations", "Profiles: People", 
                              "Profiles: Products", "Profiles", "Row Labels", "Sum of value"])
funding_df_2012

,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,total
Row Labels,,,,,,,,,,,,,
Rounds: angel,53,38,36,45,46,72,51,64,41,48,49,30,573
Rounds: seed,124,135,142,129,220,144,159,197,182,162,182,147,"1,923"
Rounds: venture,289,241,234,282,279,273,271,227,251,229,233,162,"2,971"
Rounds: convertible_note,1,NaN,NaN,1,2,NaN,2,NaN,NaN,2,1,1,10
Rounds: debt_financing,17,12,21,20,13,18,20,18,9,13,9,7,177
Rounds: equity_crowdfunding,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,2
Rounds: grant,6,7,6,16,14,14,12,14,10,9,3,3,114
Rounds: post_ipo_equity,NaN,3,1,3,1,NaN,1,NaN,1,2,1,1,14
Rounds: private_equity,7,10,6,8,7,12,9,3,6,4,11,9,92


In [169]:
funding_2013_14_csv = "funding_2013:14.csv"
funding_2013_14 = pd.read_csv(funding_2013_14_csv)

# # # dropping extraneous first two rows
funding_2013_14 = funding_2013_14.dropna(how='any') 

# # # # # making "Row Labels" as the column head
header = funding_2013_14.iloc[0]
funding_2013_14= funding_2013_14[1:] 
funding_2013_14 = funding_2013_14.rename(columns=header)
# # # # making 'row labels' as the index
funding_2013_14 = funding_2013_14.set_index("Row Labels")
funding_2013_14 = funding_2013_14.drop(["Profiles: Organizations", "Profiles: People", 
                              "Profiles: Products", "Profiles"])
funding_2013_14

,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,total
Row Labels,,,,,,,,,,,,,,
Rounds: angel,106,68,118,106,133,93,115,142,220,116,120,95,90,"1,522"
Rounds: seed,584,342,521,918,"1,368",644,"1,405",825,737,901,"1,155",808,602,"10,810"
Rounds: venture,"1,524","1,647","1,875","1,826","1,407",879,985,"1,225","1,062",740,740,636,743,"15,289"
Rounds: convertible_note,47,26,41,52,41,19,32,31,121,32,71,25,67,605
Rounds: debt_financing,361,411,361,458,292,141,125,111,204,132,106,87,141,"2,930"
Rounds: grant,57,52,49,57,40,36,131,44,109,61,82,49,27,794
Rounds: post_ipo_equity,15,10,15,35,51,19,31,31,17,31,26,19,14,314
Rounds: private_equity,42,40,44,51,36,46,74,72,40,41,46,60,38,630
Rounds: product_crowdfunding,13,16,25,10,7,4,13,11,14,13,13,10,7,156


In [57]:
rounding_csv = "rounds.csv"
rounds_df = pd.read_csv(rounding_csv)
rounds_df = rounds_df.reset_index(drop=True)
# chose columns to use except for raised_amount_usd
# error message KeyError: "['raised_amount_usd'] not in index"
rounds_df = rounds_df[['company_name','company_market',
                      'funding_round_type','funded_year',
                       'company_country_code']]

rounds_df

,company_name,company_market,funding_round_type,funded_year,company_country_code
0,#waywire,News,seed,2012.0,USA
1,&TV Communications,Games,venture,2010.0,USA
2,&TV Communications,Games,venture,2010.0,USA
3,'Rock' Your Paper,Publishing,seed,2012.0,EST
4,(In)Touch Network,Electronics,seed,2011.0,GBR
5,-R- Ranch and Mine,Tourism,equity_crowdfunding,2014.0,USA
6,-R- Ranch and Mine,Tourism,equity_crowdfunding,2014.0,USA
7,.Club Domains,Software,venture,2013.0,USA
8,.Fox Networks,Advertising,undisclosed,2007.0,ARG
9,0-6.com,Curated Web,venture,2008.0,NaN
